# New York vs. China Air Pollution

For this project, I'm going to find the four days in New York over the past four years with the highest air pollution, and see how these compare to the worst four days over past four years in Beijing, and then the city in China with the worst air-pollution.

# Data Cleaning

I accessed archived data on air quality for New York and Beijing from The World Air Quality Project, which takes its data from Chinese and US government bureaus. 

The index we will be look at is the level of pm2.5, a fine particle emitted through exhaust fumes and linked to an icnrease in respiratory and cardiovascular diseases when breathed in over a long period of time. 

In [1]:
import pandas as pd

In [2]:
df_beijing = pd.read_csv('beijing-air-quality.csv')
df_beijing

,,,,,,beijing-air-quality
date,pm25,pm10,o3,no2,so2,co
2023/6/2,105,30,53,5,,1
2023/6/3,34,28,76,11,,2
2023/6/4,41,26,65,10,,2
2023/6/5,40,53,96,11,1,3
...,...,...,...,...,...,...
2019/1/1,,46,17,20,5,9
2017/9/11,,23,35,19,1,3
2015/1/1,,67,19,29,16,11
2014/1/1,,85,20,30,14,20


In [3]:
# Unhelpfully the data comes with only the first row considered an object. We have to turn everything in 
# the second row into an object
df_beijing = pd.read_csv('beijing-air-quality.csv', header=1)
# del df_beijing['pm10']
# del df_beijing['o3']
# del df_beijing['no2']
# del df_beijing['so2']
df_beijing

,date,pm25,pm10,o3,no2,so2,co
0,2023/6/2,105,30,53,5,,1
1,2023/6/3,34,28,76,11,,2
2,2023/6/4,41,26,65,10,,2
3,2023/6/5,40,53,96,11,1,3
4,2023/6/6,74,29,71,5,1,2
...,...,...,...,...,...,...,...
3447,2019/1/1,,46,17,20,5,9
3448,2017/9/11,,23,35,19,1,3
3449,2015/1/1,,67,19,29,16,11
3450,2014/1/1,,85,20,30,14,20


In [4]:
# df_beijing.pm25.median()

In [5]:
# Oh no we still have problems! It turns out the blank spaces are considered strings by pandas, and is therefore
# getting confused about how to apply these to maths. I look around on Stack Overflow and found this solution:
import numpy as np
df_beijing = df_beijing.replace(r'^\s*$', np.nan, regex=True)
df_beijing

,date,pm25,pm10,o3,no2,so2,co
0,2023/6/2,105,30,53,5,NaN,1
1,2023/6/3,34,28,76,11,NaN,2
2,2023/6/4,41,26,65,10,NaN,2
3,2023/6/5,40,53,96,11,1,3
4,2023/6/6,74,29,71,5,1,2
...,...,...,...,...,...,...,...
3447,2019/1/1,NaN,46,17,20,5,9
3448,2017/9/11,NaN,23,35,19,1,3
3449,2015/1/1,NaN,67,19,29,16,11
3450,2014/1/1,NaN,85,20,30,14,20


In [6]:
df_beijing.pm25.median()

107.0

In [7]:
df_beijing['pm25'].sort_values(ascending = False)
# This is confusing. There are many pm25 numbers way higher than 99. I tried ascending = True and got 
# 10 and 100, but there are still numbers higher than that in the data. Same thing
# happens when we try .head. So why does it put these at the top?

1238     99
1623     99
3136     99
535      99
193      99
       ... 
3447    NaN
3448    NaN
3449    NaN
3450    NaN
3451    NaN
Name: pm25, Length: 3452, dtype: object

In [8]:
# Checked to see if removing all NaN rows would solve it...it didn't.
df_beijing.dropna(inplace=True)
df_beijing

,date,pm25,pm10,o3,no2,so2,co
3,2023/6/5,40,53,96,11,1,3
4,2023/6/6,74,29,71,5,1,2
5,2023/6/7,45,49,72,13,1,3
12,2023/6/14,23,48,120,11,2,5
13,2023/6/15,85,57,150,11,3,6
...,...,...,...,...,...,...,...
3441,2014/3/28,321,108,39,33,14,21
3442,2014/3/29,207,68,43,15,3,16
3443,2014/3/30,103,59,43,21,3,10
3444,2014/3/31,93,117,39,34,18,22


In [9]:
# How about checking to see if all numbers were saved as numbers rather than strings? That worked.
df_beijing['pm25'] = pd.to_numeric(df_beijing['pm25'], errors='coerce')
df_beijing

,date,pm25,pm10,o3,no2,so2,co
3,2023/6/5,40,53,96,11,1,3
4,2023/6/6,74,29,71,5,1,2
5,2023/6/7,45,49,72,13,1,3
12,2023/6/14,23,48,120,11,2,5
13,2023/6/15,85,57,150,11,3,6
...,...,...,...,...,...,...,...
3441,2014/3/28,321,108,39,33,14,21
3442,2014/3/29,207,68,43,15,3,16
3443,2014/3/30,103,59,43,21,3,10
3444,2014/3/31,93,117,39,34,18,22


In [10]:
# We'd better convert those dates to numerical dates too...
df_beijing['date'] = pd.to_datetime(df_beijing['date'], format='%Y/%m/%d')
df_beijing


,date,pm25,pm10,o3,no2,so2,co
3,2023-06-05,40,53,96,11,1,3
4,2023-06-06,74,29,71,5,1,2
5,2023-06-07,45,49,72,13,1,3
12,2023-06-14,23,48,120,11,2,5
13,2023-06-15,85,57,150,11,3,6
...,...,...,...,...,...,...,...
3441,2014-03-28,321,108,39,33,14,21
3442,2014-03-29,207,68,43,15,3,16
3443,2014-03-30,103,59,43,21,3,10
3444,2014-03-31,93,117,39,34,18,22


In [11]:
cutoff_date = pd.to_datetime('2020-01-01')
df_recent = df_beijing[df_beijing['date'] > cutoff_date]

In [12]:
df_recent.sort_values(by = 'pm25', ascending = False).head(20)
# Ok, so as expected there a lot of days where the air pollution in BJ was WAAAAY worse than it was in NY....
# But...predictable correlation here between days with high pm25 and pandemic public holidays and weekends:
# 7/10 of the top ten are on/day before a holiday or on a weekend 
# spring festival: February 12th 2021, January 25th 2020...I tried following up on this 

,date,pm25,pm10,o3,no2,so2,co
850,2021-02-13,273,132,13,21,2,22
1193,2020-01-19,251,58,28,10,1,4
1203,2020-01-29,245,38,33,16,5,11
876,2021-03-11,242,66,43,23,1,7
849,2021-02-12,239,139,50,17,5,15
145,2023-03-08,237,101,47,21,1,7
63,2023-04-14,224,189,41,14,2,4
220,2022-11-21,219,52,21,14,1,4
143,2023-03-06,211,123,49,23,1,9
870,2021-03-05,209,48,21,24,1,12


In [13]:
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2023-06-20')
weekend_dates = []
for date in pd.date_range(start_date, end_date):
    date = date.strftime("%Y-%m-%d")
    if date>5:
        list.append(date)
print(weekend_date)

TypeError: '>' not supported between instances of 'str' and 'int'

In [14]:
Date = pd.bdate_range(start="2020-01-01", end="2023-06-20", freq="C", weekmask="Sat Sun")
Date

DatetimeIndex(['2020-01-04', '2020-01-05', '2020-01-11', '2020-01-12',
               '2020-01-18', '2020-01-19', '2020-01-25', '2020-01-26',
               '2020-02-01', '2020-02-02',
               ...
               '2023-05-20', '2023-05-21', '2023-05-27', '2023-05-28',
               '2023-06-03', '2023-06-04', '2023-06-10', '2023-06-11',
               '2023-06-17', '2023-06-18'],
              dtype='datetime64[ns]', length=362, freq='C')

In [15]:
df_recent['weekend'] = np.where(df_recent['date'].isin(Date),'True', 'False')

/var/folders/hj/vgz_33d52cb3y_xnhk51g0rm0000gn/T/ipykernel_25782/1498565630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recent['weekend'] = np.where(df_recent['date'].isin(Date),'True', 'False')


In [16]:
df_recent.loc['weekend'] = False

/var/folders/hj/vgz_33d52cb3y_xnhk51g0rm0000gn/T/ipykernel_25782/1621465146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recent.loc['weekend'] = False


In [17]:
df_recent.loc[df_recent['date'].isin(Date), 'weekend'] = 'True'
df_recent.loc[~df_recent['date'].isin(Date), 'weekend'] = 'False'
df_recent

,date,pm25,pm10,o3,no2,so2,co,weekend
3,2023-06-05 00:00:00,40,53,96,11,1,3,False
4,2023-06-06 00:00:00,74,29,71,5,1,2,False
5,2023-06-07 00:00:00,45,49,72,13,1,3,False
12,2023-06-14 00:00:00,23,48,120,11,2,5,False
13,2023-06-15 00:00:00,85,57,150,11,3,6,False
...,...,...,...,...,...,...,...,...
1261,2020-03-27 00:00:00,39,25,33,4,1,2,False
1263,2020-03-29 00:00:00,37,63,42,17,3,4,True
1264,2020-03-30 00:00:00,122,69,42,16,3,6,False
1265,2020-03-31 00:00:00,156,62,45,12,2,3,False


In [19]:
df_recent.sort_values(by = 'pm25', ascending = False).head(50)

,date,pm25,pm10,o3,no2,so2,co,weekend
850,2021-02-13 00:00:00,273,132,13,21,2,22,True
1193,2020-01-19 00:00:00,251,58,28,10,1,4,True
1203,2020-01-29 00:00:00,245,38,33,16,5,11,False
876,2021-03-11 00:00:00,242,66,43,23,1,7,False
849,2021-02-12 00:00:00,239,139,50,17,5,15,False
145,2023-03-08 00:00:00,237,101,47,21,1,7,False
63,2023-04-14 00:00:00,224,189,41,14,2,4,False
220,2022-11-21 00:00:00,219,52,21,14,1,4,False
143,2023-03-06 00:00:00,211,123,49,23,1,9,False
870,2021-03-05 00:00:00,209,48,21,24,1,12,False


In [ ]:
# So no correlation between worst levels of pm2.5 vs. weekends and public holidays....although this could be an 
# interesting thing to follow up on in the future on the RELATION between each weekend and its surrounding days. 

In [ ]:
# I opened up the air quality records for Huludao, a city with famously bad air quality...

In [20]:
df = pd.read_csv('huludao-air-quality.csv')

In [21]:
df.columns

Index(['date', ' pm25', ' pm10', ' o3', ' no2', ' so2', ' co'], dtype='object')

In [22]:
# df.sort_values(by = 'pm25')

In [23]:
df.columns = [x.strip() for x in df.columns]
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,date,pm25,pm10,o3,no2,so2,co
0,2023/6/2,130,34,53,10,9,2
1,2023/6/3,48,38,59,16,7,2
2,2023/6/4,59,46,97,16,14,4
3,2023/6/5,74,58,103,12,10,6
4,2023/6/6,135,46,85,8,10,4
...,...,...,...,...,...,...,...
3399,2017/9/11,NaN,49,39,16,33,11
3400,2017/1/2,NaN,267,35,47,92,45
3401,2016/1/4,NaN,103,33,31,63,34
3402,2015/1/1,NaN,61,17,18,48,25


In [24]:
df.dropna(inplace = True)
df

,date,pm25,pm10,o3,no2,so2,co
0,2023/6/2,130,34,53,10,9,2
1,2023/6/3,48,38,59,16,7,2
2,2023/6/4,59,46,97,16,14,4
3,2023/6/5,74,58,103,12,10,6
4,2023/6/6,135,46,85,8,10,4
...,...,...,...,...,...,...,...
3389,2014/3/27,174,94,70,27,53,29
3390,2014/3/28,159,105,55,32,33,21
3391,2014/3/29,173,69,56,21,30,20
3392,2014/3/30,134,101,76,41,39,22


In [25]:
df.sort_values(by = 'pm25', ascending = True)

,date,pm25,pm10,o3,no2,so2,co
1930,2018/3/17,100,55,36,29,39,19
1916,2018/3/3,100,85,33,17,36,13
1898,2018/2/13,100,98,29,27,68,28
1943,2018/3/30,100,90,72,27,55,12
2030,2017/12/25,100,60,32,20,38,12
...,...,...,...,...,...,...,...
2192,2017/6/7,99,44,98,10,21,12
1839,2018/6/15,99,39,72,10,3,8
1463,2019/6/2,99,61,96,9,5,9
682,2021/8/24,99,28,47,7,5,7


In [26]:
df['pm25'] = pd.to_numeric(df['pm25'], errors='coerce')
df

,date,pm25,pm10,o3,no2,so2,co
0,2023/6/2,130,34,53,10,9,2
1,2023/6/3,48,38,59,16,7,2
2,2023/6/4,59,46,97,16,14,4
3,2023/6/5,74,58,103,12,10,6
4,2023/6/6,135,46,85,8,10,4
...,...,...,...,...,...,...,...
3389,2014/3/27,174,94,70,27,53,29
3390,2014/3/28,159,105,55,32,33,21
3391,2014/3/29,173,69,56,21,30,20
3392,2014/3/30,134,101,76,41,39,22


In [27]:
df.sort_values(by = 'pm25', ascending = False).head(50)
# I'm glad I never visited Huludao. 

,date,pm25,pm10,o3,no2,so2,co
1153,2020/1/26,532,170,31,19,52,22
1504,2019/1/13,396,264,22,40,77,51
833,2021/1/26,392,208,23,38,45,30
2384,2016/12/19,376,198,32,34,52,26
1505,2019/1/14,371,139,15,36,48,25
1308,2019/12/29,362,222,14,26,63,25
2250,2017/2/5,360,101,38,16,52,13
2249,2017/2/4,358,210,30,34,55,36
2752,2015/12/24,357,85,35,25,63,27
3350,2014/2/16,345,101,46,26,72,25


In [28]:
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df

,date,pm25,pm10,o3,no2,so2,co
0,2023-06-02,130,34,53,10,9,2
1,2023-06-03,48,38,59,16,7,2
2,2023-06-04,59,46,97,16,14,4
3,2023-06-05,74,58,103,12,10,6
4,2023-06-06,135,46,85,8,10,4
...,...,...,...,...,...,...,...
3389,2014-03-27,174,94,70,27,53,29
3390,2014-03-28,159,105,55,32,33,21
3391,2014-03-29,173,69,56,21,30,20
3392,2014-03-30,134,101,76,41,39,22


In [29]:
cutoff_date = pd.to_datetime('2020-01-01')
df = df[df['date'] > cutoff_date]
df

,date,pm25,pm10,o3,no2,so2,co
0,2023-06-02,130,34,53,10,9,2
1,2023-06-03,48,38,59,16,7,2
2,2023-06-04,59,46,97,16,14,4
3,2023-06-05,74,58,103,12,10,6
4,2023-06-06,135,46,85,8,10,4
...,...,...,...,...,...,...,...
1215,2020-03-28,64,55,33,16,12,6
1216,2020-03-29,68,63,40,18,11,8
1217,2020-03-30,106,60,49,15,10,8
1218,2020-03-31,120,80,48,12,13,8


In [30]:
df.sort_values(by = 'date', ascending = False)

,date,pm25,pm10,o3,no2,so2,co
19,2023-06-21,40,36,65,13,9,4
18,2023-06-20,120,14,54,4,2,3
17,2023-06-19,120,46,72,6,3,4
16,2023-06-18,96,53,76,8,3,4
15,2023-06-17,94,48,87,6,4,4
...,...,...,...,...,...,...,...
1133,2020-01-06,239,94,12,29,28,18
1132,2020-01-05,237,138,13,36,36,24
1131,2020-01-04,323,152,23,33,71,27
1130,2020-01-03,234,246,13,45,67,32


In [31]:
# I now try to extract the worst day for each year 2020-2023 from Huludao, Beijing, and New York, then 
# concatenate them all together...

df.dtypes


date    datetime64[ns]
pm25             int64
pm10            object
o3              object
no2             object
so2             object
co              object
dtype: object

In [32]:
df['Month'] = df['date'].dt.month

/var/folders/hj/vgz_33d52cb3y_xnhk51g0rm0000gn/T/ipykernel_25782/4024458858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['date'].dt.month


In [33]:
df['year'] = df['date'].dt.year 

/var/folders/hj/vgz_33d52cb3y_xnhk51g0rm0000gn/T/ipykernel_25782/3902704605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['date'].dt.year


In [34]:
df = df[['date','pm25', 'year']]

In [35]:
df_2020 = df[df["year"] == 2020]

In [36]:
df_2020.sort_values(by="pm25", ascending=False).head(10)

,date,pm25,year
1153,2020-01-26,532,2020
1131,2020-01-04,323,2020
1154,2020-01-27,304,2020
1146,2020-01-19,282,2020
1133,2020-01-06,239,2020
1132,2020-01-05,237,2020
1130,2020-01-03,234,2020
1138,2020-01-11,232,2020
1145,2020-01-18,229,2020
961,2020-12-03,222,2020


In [37]:
df_2020 = df_2020[df_2020["pm25"] == "532"]

In [38]:
df_2021 = df[df["year"] == 2021]
df_2021.sort_values(by = "pm25", ascending = False)

,date,pm25,year
833,2021-01-26,392,2021
829,2021-01-22,331,2021
834,2021-01-27,326,2021
832,2021-01-25,311,2021
860,2021-02-22,274,2021
...,...,...,...
538,2021-10-05,46,2021
683,2021-08-25,45,2021
659,2021-08-01,44,2021
697,2021-09-08,31,2021


In [39]:
df_2021 = df_2021[df_2021["pm25"] == 392]

In [40]:
df_2022 = df[df["year"] == 2022]
df_2022.sort_values(by = "pm25", ascending = False)

,date,pm25,year
313,2022-08-20,243,2022
485,2022-02-11,233,2022
503,2022-03-01,226,2022
451,2022-01-08,219,2022
211,2022-11-11,208,2022
...,...,...,...
326,2022-09-02,39,2022
187,2022-10-18,37,2022
348,2022-09-24,35,2022
179,2022-10-10,35,2022


In [41]:
df_2022 = df_2022[df_2022["pm25"] == 243]

In [42]:
df_2023 = df[df["year"] == 2023]
df_2023.sort_values(by='pm25', ascending = False)

,date,pm25,year
143,2023-03-05,250,2023
85,2023-01-06,234,2023
86,2023-01-07,217,2023
122,2023-02-12,207,2023
169,2023-03-31,195,2023
...,...,...,...
58,2023-04-08,56,2023
1,2023-06-03,48,2023
26,2023-05-07,47,2023
94,2023-01-15,44,2023


In [43]:
df_2023 = df_2023[df_2023["pm25"] == 250]

In [44]:
df_bj = df_recent
df_bj

,date,pm25,pm10,o3,no2,so2,co,weekend
3,2023-06-05 00:00:00,40,53,96,11,1,3,False
4,2023-06-06 00:00:00,74,29,71,5,1,2,False
5,2023-06-07 00:00:00,45,49,72,13,1,3,False
12,2023-06-14 00:00:00,23,48,120,11,2,5,False
13,2023-06-15 00:00:00,85,57,150,11,3,6,False
...,...,...,...,...,...,...,...,...
1261,2020-03-27 00:00:00,39,25,33,4,1,2,False
1263,2020-03-29 00:00:00,37,63,42,17,3,4,True
1264,2020-03-30 00:00:00,122,69,42,16,3,6,False
1265,2020-03-31 00:00:00,156,62,45,12,2,3,False


In [77]:
df_bj['year'] = df_bj['date'].dt.year
df_bj = df_bj[['date','pm25','year']]
df_bj
# This piece of code, indeed all others with errors, all worked previously, in this case creating a new column 
# with the year.

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
df_bj_2023 = df_bj[df_bj["year"] == 2023]
df_bj_2023.sort_values(by = "pm25", ascending = False)

In [46]:
df_bj_2023 = df_bj_2023[df_bj_2023["pm25"] == 237]

NameError: name 'df_bj_2023' is not defined

In [47]:
df_bj_2022 = df_bj[df_bj["year"] == 2022]
df_bj_2022.sort_values(by = "pm25", ascending = False)

KeyError: 'year'

In [ ]:
df_bj_2022 = df_bj_2022[df_bj_2022["pm25"] == 219]

In [ ]:
df_bj_2021 = df_bj[df_bj["year"] == 2021]
df_bj_2021.sort_values(by = "pm25", ascending = False)

In [48]:
df_bj_2021 = df_bj_2021[df_bj_2021["pm25"] == 273]

NameError: name 'df_bj_2021' is not defined

In [49]:
df_bj_2020 = df_bj[df_bj["year"] == 2020]
df_bj_2020.sort_values(by = "pm25", ascending = False)

KeyError: 'year'

In [50]:
df_bj_2020 = df_bj_2020[df_bj_2020["pm25"] == 251]

NameError: name 'df_bj_2020' is not defined

In [51]:
df = pd.read_csv('new-york, usa-air-quality.csv', header = 1)
df = df.replace(r'^\s*$', np.nan, regex=True)

In [52]:
df.dropna(inplace = True)
df

,date,pm25,o3,no2,co
0,2023/6/2,62,72,25,4
1,2023/6/3,65,27,6,1
2,2023/6/4,52,26,3,1
3,2023/6/5,22,32,5,1
4,2023/6/6,37,43,10,4
...,...,...,...,...,...
1857,2018/6/25,64,30,8,1
1858,2018/6/26,31,28,6,1
1859,2018/6/27,19,30,6,1
1862,2018/6/30,35,66,15,3


In [53]:
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
cutoff_date = pd.to_datetime('2020-01-01')
df = df[df['date'] > cutoff_date]
df

,date,pm25,o3,no2,co
0,2023-06-02,62,72,25,4
1,2023-06-03,65,27,6,1
2,2023-06-04,52,26,3,1
3,2023-06-05,22,32,5,1
4,2023-06-06,37,43,10,4
...,...,...,...,...,...
1160,2020-06-26,37,44,25,1
1162,2020-06-28,56,50,7,1
1163,2020-06-29,45,32,8,1
1164,2020-06-30,21,34,6,1


In [54]:
df['pm25'] = pd.to_numeric(df['pm25'], errors='coerce')
df

/var/folders/hj/vgz_33d52cb3y_xnhk51g0rm0000gn/T/ipykernel_25782/1920415525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pm25'] = pd.to_numeric(df['pm25'], errors='coerce')


,date,pm25,o3,no2,co
0,2023-06-02,62,72,25,4
1,2023-06-03,65,27,6,1
2,2023-06-04,52,26,3,1
3,2023-06-05,22,32,5,1
4,2023-06-06,37,43,10,4
...,...,...,...,...,...
1160,2020-06-26,37,44,25,1
1162,2020-06-28,56,50,7,1
1163,2020-06-29,45,32,8,1
1164,2020-06-30,21,34,6,1


In [55]:
df_ny = df
df_ny['year'] = df_ny['date'].dt.year
df_ny = df_ny[['date', 'pm25', 'year']]
df_ny

/var/folders/hj/vgz_33d52cb3y_xnhk51g0rm0000gn/T/ipykernel_25782/1309405529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ny['year'] = df_ny['date'].dt.year


,date,pm25,year
0,2023-06-02,62,2023
1,2023-06-03,65,2023
2,2023-06-04,52,2023
3,2023-06-05,22,2023
4,2023-06-06,37,2023
...,...,...,...
1160,2020-06-26,37,2020
1162,2020-06-28,56,2020
1163,2020-06-29,45,2020
1164,2020-06-30,21,2020


In [56]:
df_ny.sort_values(by='pm25', ascending = False)

,date,pm25,year
6,2023-06-08,247,2023
7,2023-06-09,168,2023
5,2023-06-07,163,2023
701,2021-09-23,123,2021
696,2021-09-18,122,2021
...,...,...,...
1059,2020-09-16,10,2020
787,2021-06-18,9,2021
426,2022-06-20,9,2022
992,2020-07-11,8,2020


In [57]:
df_ny_2023 = df_ny[df_ny["year"] == 2023]
df_ny_2023.sort_values(by = "pm25", ascending = False)

,date,pm25,year
6,2023-06-08,247,2023
7,2023-06-09,168,2023
5,2023-06-07,163,2023
10,2023-06-12,91,2023
16,2023-06-18,83,2023
...,...,...,...
52,2023-04-03,18,2023
22,2023-05-04,17,2023
21,2023-05-03,17,2023
74,2023-04-25,15,2023


In [58]:
df_ny_2023 = df_ny_2023[df_ny_2023["year"] == 247]

In [59]:
df_ny_2022 = df_ny[df_ny["year"] == 2022]
df_ny_2022.sort_values(by = "pm25", ascending = False)

,date,pm25,year
445,2022-01-11,111,2022
615,2022-01-01,90,2022
259,2022-12-31,85,2022
446,2022-01-12,79,2022
352,2022-04-07,75,2022
...,...,...,...
399,2022-05-24,12,2022
423,2022-06-17,12,2022
344,2022-09-24,10,2022
268,2022-07-08,10,2022


In [60]:
df_ny_2022 = df_ny_2022[df_ny_2022["pm25"] == 111]

In [61]:
df_ny_2021 = df_ny[df_ny["year"] == 2021]
df_ny_2021.sort_values(by = "pm25", ascending = False)

,date,pm25,year
701,2021-09-23,123,2021
696,2021-09-18,122,2021
687,2021-09-09,122,2021
688,2021-09-10,121,2021
684,2021-09-06,120,2021
...,...,...,...
739,2021-05-01,14,2021
793,2021-06-24,12,2021
786,2021-06-17,10,2021
787,2021-06-18,9,2021


In [62]:
df_ny_2021 = df_ny_2021[df_ny_2021["pm25"] == 123]

In [63]:
df_ny_2020 = df_ny[df_ny["year"] == 2020]
df_ny_2020.sort_values(by = "pm25", ascending = False)

,date,pm25,year
987,2020-07-06,63,2020
1162,2020-06-28,56,2020
988,2020-07-07,56,2020
986,2020-07-05,55,2020
1017,2020-08-05,54,2020
...,...,...,...
1148,2020-06-14,11,2020
1105,2020-05-02,11,2020
1113,2020-05-10,11,2020
1059,2020-09-16,10,2020


In [64]:
df_ny_2020 = df_ny_2020[df_ny_2020["pm25"] == 63]
df_ny_2020

,date,pm25,year
987,2020-07-06,63,2020


In [65]:
merged = df_ny_2020.merge(df_bj_2020, how="outer", on="year")

NameError: name 'df_bj_2020' is not defined

In [66]:
merged

NameError: name 'merged' is not defined

In [67]:
df_bj_2020

NameError: name 'df_bj_2020' is not defined

In [68]:
df_ny_2020

,date,pm25,year
987,2020-07-06,63,2020


In [69]:
concatenated = pd.concat([df_bj_2020, df_ny_2020, df_bj_2021])

NameError: name 'df_bj_2020' is not defined

In [70]:
# dfs = [df_ny_2020, df_bj_2020, df_2020, df_ny_2021, df_bj_2021, df_2021, df_ny_2022, df_bj_2022, df_2022, df_ny_2023, df_bj_2023, df_2023]
# for df in dfs:
#     df.to_dict('list')
# dfs

In [71]:
concatenated

NameError: name 'concatenated' is not defined

In [72]:
# I couldn't get pandas to concatenate beyond the first two. Time constraints meant I had to cheat. 

In [73]:
df = pd.DataFrame({
'pm25': [532, 251, 63, 392, 273, 123, 243, 219, 111, 250, 237, 247],
                   'year': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022, 2023, 2023, 2023]})
df

,pm25,year
0,532,2020
1,251,2020
2,63,2020
3,392,2021
4,273,2021
5,123,2021
6,243,2022
7,219,2022
8,111,2022
9,250,2023


In [74]:
df.to_csv('pollution.csv', sep=',', index=False, encoding='utf-8')